In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from numpy import load
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.applications.resnet import ResNet101 


In [3]:

 

def load_dataset():

	data = load('/content/drive/MyDrive/Visión Artificial/Sol_20201121/Final/planet_data_64_all_train2.npz')
	X, y = data['arr_0'], data['arr_1']
	return X, y
 

def define_model(in_shape=(64, 64, 3), out_shape=17):

	model = ResNet101(include_top=False, input_shape=in_shape) 

	for layer in model.layers:
		layer.trainable = False

	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(out_shape, activation='sigmoid')(class1)

	model = Model(inputs=model.inputs, outputs=output)

	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy')
	return model
 

def run_test_harness():

	X, y = load_dataset()

	datagen = ImageDataGenerator(featurewise_center=True, horizontal_flip=True, vertical_flip=True, rotation_range=90)

	train_it = datagen.flow(X, y, batch_size=128)

	model = define_model()

	model.fit_generator(train_it, steps_per_epoch=len(train_it), epochs=50, verbose=0)

	model.save('/content/drive/MyDrive/Visión Artificial/Sol_20201121/Final/mf_resnet101.h5')
 

run_test_harness()


KeyboardInterrupt: ignored

In [3]:
import sys
from numpy import load
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras import backend
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
 

def load_dataset():

	data = load('/content/drive/MyDrive/Visión Artificial/Sol_20201121/Final/planet_data_64_all_train2.npz')
	X, y = data['arr_0'], data['arr_1']

	trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.3, random_state=1)
	print(trainX.shape, trainY.shape, testX.shape, testY.shape)
	return trainX, trainY, testX, testY
 

def fbeta(y_true, y_pred, beta=2):

	y_pred = backend.clip(y_pred, 0, 1)

	tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
	fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
	fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)

	p = tp / (tp + fp + backend.epsilon())

	r = tp / (tp + fn + backend.epsilon())

	bb = beta ** 2
	fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
	return fbeta_score
 

def define_model(in_shape=(64, 64, 3), out_shape=17):

	model = ResNet101(include_top=False, input_shape=in_shape) #VGG16(include_top=False, input_shape=in_shape)

	for layer in model.layers:
		layer.trainable = False

	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(out_shape, activation='sigmoid')(class1)

	model = Model(inputs=model.inputs, outputs=output)

	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[fbeta])
	return model
 

def summarize_diagnostics(history):

	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')

	pyplot.subplot(212)
	pyplot.title('Fbeta')
	pyplot.plot(history.history['fbeta'], color='blue', label='train')
	pyplot.plot(history.history['val_fbeta'], color='orange', label='test')

	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()
 

def run_test_harness():

	trainX, trainY, testX, testY = load_dataset()

	train_datagen = ImageDataGenerator(featurewise_center=True, horizontal_flip=True, vertical_flip=True, rotation_range=90)
	test_datagen = ImageDataGenerator(featurewise_center=True)


	train_it = train_datagen.flow(trainX, trainY, batch_size=128)
	test_it = test_datagen.flow(testX, testY, batch_size=128)

	model = define_model()

	history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=50, verbose=0)

	loss, fbeta = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
	print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))

	summarize_diagnostics(history)
 

run_test_harness()

(28327, 64, 64, 3) (28327, 17) (12141, 64, 64, 3) (12141, 17)
171450368/171446536 [==============================] - 3s 0us/step
Instructions for updating:
Please use Model.fit, which supports generators.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Instructions for updating:
Please use Model.evaluate, which supports generators.
> loss=0.123, fbeta=0.845
